Question 1- Is there any correlation between physical strength and fearfulness (try to compare pearson and spearman)? (Farnaz and Kavleen)

Question 2- Is the relationship between physical strength and fearfulness predictable? (Absence of correlation does not mean that we cannot predict target (specially if non-linear))

Question 3- What are the important features in the prediction of fearfulness? (Niki)

Question 4- Can we predict a fearfulness score of a participant based on other features in the dataset? (Maryam:SVM) (Niki:RF)

In [79]:
### Libraries used
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, col, expr, rand, count, when, isnan
# from pyspark.ml.classification import *
from pyspark.ml.regression import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, Imputer
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np

In [80]:
###Read Dataset
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

spark = init_spark()
df = spark.read.csv('Sample_1.csv', inferSchema =True, header=True)
df

DataFrame[p_id: int, age: int, female: int, ethnicity: int, grip: int, chest: string, e_anx_1: int, e_anx_2: int, e_anx_3: int, e_anx_4: int, e_anx_5: int, e_anx_6: int, e_anx_7: int, e_anx_8: int, e_anx_9: int, e_anx_10: int, e_dep_1: int, e_dep_2: int, e_dep_3: int, e_dep_4: int, e_dep_5: int, e_dep_6: int, e_dep_7: int, e_dep_8: int, e_dep_9: int, e_dep_10: int, e_fear_1: int, e_fear_2: int, e_fear_3: int, e_fear_4: int, e_fear_5: int, e_fear_6: int, e_fear_7: int, e_fear_8: int, e_fear_9: int, e_fear_10: int, e_sen_1: int, e_sen_2: int, e_sen_3: int, e_sen_4: int, e_sen_5: int, e_sen_6: int, e_sen_7: int, e_sen_8: int, e_sen_9: int, e_sen_10: int]

# Data Preparation

In [81]:
anx_cols_to_average = ['e_anx_1', 'e_anx_2', 'e_anx_3','e_anx_4', 'e_anx_5', 'e_anx_6','e_anx_7', 'e_anx_8', 'e_anx_9', 'e_anx_10']
dep_cols_to_average = ['e_dep_1', 'e_dep_2', 'e_dep_3','e_dep_4', 'e_dep_5', 'e_dep_6','e_dep_7', 'e_dep_8', 'e_dep_9', 'e_dep_10']
fear_cols_to_average = ['e_fear_1', 'e_fear_2', 'e_fear_3','e_fear_4', 'e_fear_5', 'e_fear_6','e_fear_7', 'e_fear_8', 'e_fear_9', 'e_fear_10']
sen_cols_to_average = ['e_sen_1', 'e_sen_2', 'e_sen_3','e_sen_4', 'e_sen_5', 'e_sen_6','e_sen_7', 'e_sen_8', 'e_sen_9', 'e_sen_10']

mean_anx  = (sum(col(x) for x in anx_cols_to_average) / 10).alias("mean_anx")
mean_dep  = (sum(col(x) for x in dep_cols_to_average) / 10).alias("mean_dep")
mean_fear  = (sum(col(x) for x in fear_cols_to_average) / 10).alias("mean_fear")
mean_sen  = (sum(col(x) for x in sen_cols_to_average) / 10).alias("mean_sen")

df_with_means = df.withColumn("mean_anx", mean_anx).withColumn("mean_dep",mean_dep).withColumn("mean_fear",mean_fear).withColumn("mean_sen",mean_sen)

cols_to_drop = ['e_anx_1', 'e_anx_2', 'e_anx_3','e_anx_4', 'e_anx_5', 'e_anx_6','e_anx_7', 'e_anx_8', 'e_anx_9', 'e_anx_10',\
          'e_dep_1', 'e_dep_2', 'e_dep_3','e_dep_4', 'e_dep_5', 'e_dep_6','e_dep_7', 'e_dep_8', 'e_dep_9', 'e_dep_10',\
          'e_fear_1', 'e_fear_2', 'e_fear_3','e_fear_4', 'e_fear_5', 'e_fear_6','e_fear_7', 'e_fear_8', 'e_fear_9', 'e_fear_10',\
          'e_sen_1', 'e_sen_2', 'e_sen_3','e_sen_4', 'e_sen_5', 'e_sen_6','e_sen_7', 'e_sen_8', 'e_sen_9', 'e_sen_10']
df_with_means = df_with_means.drop(*cols_to_drop)
df_with_means.collect()

[Row(p_id=103, age=21, female=0, ethnicity=8, grip=52, chest='31', mean_anx=4.9, mean_dep=4.0, mean_fear=3.7, mean_sen=3.6),
 Row(p_id=183, age=20, female=0, ethnicity=8, grip=34, chest='38', mean_anx=4.0, mean_dep=4.1, mean_fear=4.0, mean_sen=3.6),
 Row(p_id=177, age=19, female=0, ethnicity=1, grip=48, chest='32', mean_anx=4.7, mean_dep=4.1, mean_fear=3.6, mean_sen=3.5),
 Row(p_id=133, age=19, female=0, ethnicity=1, grip=48, chest='30', mean_anx=4.1, mean_dep=3.8, mean_fear=3.6, mean_sen=4.2),
 Row(p_id=171, age=18, female=0, ethnicity=2, grip=40, chest='40', mean_anx=4.7, mean_dep=4.0, mean_fear=3.7, mean_sen=3.3),
 Row(p_id=167, age=20, female=0, ethnicity=8, grip=49, chest='36', mean_anx=3.7, mean_dep=4.1, mean_fear=3.3, mean_sen=4.3),
 Row(p_id=164, age=18, female=0, ethnicity=2, grip=36, chest='40', mean_anx=3.6, mean_dep=5.1, mean_fear=4.7, mean_sen=4.0),
 Row(p_id=184, age=19, female=0, ethnicity=8, grip=38, chest='36', mean_anx=4.2, mean_dep=4.1, mean_fear=4.0, mean_sen=4.0),


In [82]:
# Covert values in the 'chest' column to int
df_with_means = df_with_means.withColumn("chest-clean", col("chest").cast("int")).drop("chest")
    
# Clean data: Fill missing values in the column 'chest'
# with the mean value in that column
imputer = Imputer(
    inputCols=df_with_means.columns,
    outputCols=["{}_imputed".format(c) for c in df_with_means.columns]
)
df_with_means = imputer.fit(df_with_means).transform(df_with_means)

# Drop original non-imputed columns
col_drop=['p_id','mean_fear', 'age', 'female', 'ethnicity', 'grip', 'chest-clean', 'mean_anx', 'mean_dep', 'mean_sen']
df_with_means = df_with_means.drop(*col_drop)
df_with_means.collect()

[Row(p_id_imputed=103, age_imputed=21, female_imputed=0, ethnicity_imputed=8, grip_imputed=52, mean_anx_imputed=4.9, mean_dep_imputed=4.0, mean_fear_imputed=3.7, mean_sen_imputed=3.6, chest-clean_imputed=31),
 Row(p_id_imputed=183, age_imputed=20, female_imputed=0, ethnicity_imputed=8, grip_imputed=34, mean_anx_imputed=4.0, mean_dep_imputed=4.1, mean_fear_imputed=4.0, mean_sen_imputed=3.6, chest-clean_imputed=38),
 Row(p_id_imputed=177, age_imputed=19, female_imputed=0, ethnicity_imputed=1, grip_imputed=48, mean_anx_imputed=4.7, mean_dep_imputed=4.1, mean_fear_imputed=3.6, mean_sen_imputed=3.5, chest-clean_imputed=32),
 Row(p_id_imputed=133, age_imputed=19, female_imputed=0, ethnicity_imputed=1, grip_imputed=48, mean_anx_imputed=4.1, mean_dep_imputed=3.8, mean_fear_imputed=3.6, mean_sen_imputed=4.2, chest-clean_imputed=30),
 Row(p_id_imputed=171, age_imputed=18, female_imputed=0, ethnicity_imputed=2, grip_imputed=40, mean_anx_imputed=4.7, mean_dep_imputed=4.0, mean_fear_imputed=3.7, me

# Q3: Feature Importance

In [43]:
train_data, test_data = df_with_means.randomSplit([0.8, 0.2], seed=123)

train_data = train_data.withColumn("label", train_data["mean_fear_imputed"])

inputCols=['age_imputed', 'female_imputed', 'ethnicity_imputed', 'grip_imputed', 'chest-clean_imputed', 'mean_anx_imputed', 'mean_dep_imputed', "mean_sen_imputed"],
outputCol='features'

assembler = VectorAssembler(
    inputCols=['age_imputed', 'female_imputed', 'ethnicity_imputed', 'grip_imputed', 'chest-clean_imputed', 'mean_anx_imputed', 'mean_dep_imputed', "mean_sen_imputed"],
    outputCol='features')

train_data_trans = assembler.transform(train_data)

## SVM

In [11]:
lr = LinearRegression(maxIter=10, regParam=0.1, elasticNetParam=0.8)
lr_model = lr.fit(train_data_trans)

# Get the feature importance
coefficients = lr_model.coefficients
print("Feature Importances: ", coefficients)
# feature_importance = [(col, coef) for col, coef in zip(inputCols, coefficients)]

# # Print the feature importance
# print("Feature Importance:")
# for feature in feature_importance:
#     print(feature[0], ": ", feature[1])

23/03/15 01:26:14 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/03/15 01:26:14 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


Feature Importances:  [0.0,0.0,0.0,0.0,0.0,0.0,0.057318983950542496,0.0]


## Random Forest

In [12]:
# Train a Random Forest model
rf = RandomForestRegressor()
rf_model = rf.fit(train_data_trans)

# Get the feature importance
rf_model.featureImportances

SparseVector(8, {0: 0.0454, 1: 0.0105, 2: 0.0442, 3: 0.134, 4: 0.1111, 5: 0.2067, 6: 0.2404, 7: 0.2078})

# Q4: Prediction

## Random Forest

In [13]:
test_data = test_data.withColumn("label", test_data["mean_fear_imputed"])
test_data_trans = assembler.transform(test_data)

# Make predictions.
predictions = model.transform(test_data_trans)

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label", metricName="mse")
mse = evaluator.evaluate(predictions)
print("Mean squared error = %g" % mse)

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R-squared = %g" % r2)

NameError: name 'model' is not defined

## SVM


In [83]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder


# Convert fearfulness scores into three categories
threshold1 = df_with_means.approxQuantile('mean_fear_imputed', [0.33], 0)[0]
threshold2 = df_with_means.approxQuantile('mean_fear_imputed', [0.66], 0)[0]
df_with_means = df_with_means.withColumn('fearfulness_category', 
                       when(col('mean_fear_imputed') <= threshold1, 'low')
                       .when((col('mean_fear_imputed') > threshold1) & (col('mean_fear_imputed') <= threshold2), 'medium')
                       .otherwise('high'))

# Covert values in the 'fearfulness_category' column to int
#df_with_means = df_with_means.withColumn("fearfulness_category_clean", col("fearfulness_category").cast("int")).drop("fearfulness_category")
df_with_means.collect()


[Row(p_id_imputed=103, age_imputed=21, female_imputed=0, ethnicity_imputed=8, grip_imputed=52, mean_anx_imputed=4.9, mean_dep_imputed=4.0, mean_fear_imputed=3.7, mean_sen_imputed=3.6, chest-clean_imputed=31, fearfulness_category='medium'),
 Row(p_id_imputed=183, age_imputed=20, female_imputed=0, ethnicity_imputed=8, grip_imputed=34, mean_anx_imputed=4.0, mean_dep_imputed=4.1, mean_fear_imputed=4.0, mean_sen_imputed=3.6, chest-clean_imputed=38, fearfulness_category='high'),
 Row(p_id_imputed=177, age_imputed=19, female_imputed=0, ethnicity_imputed=1, grip_imputed=48, mean_anx_imputed=4.7, mean_dep_imputed=4.1, mean_fear_imputed=3.6, mean_sen_imputed=3.5, chest-clean_imputed=32, fearfulness_category='medium'),
 Row(p_id_imputed=133, age_imputed=19, female_imputed=0, ethnicity_imputed=1, grip_imputed=48, mean_anx_imputed=4.1, mean_dep_imputed=3.8, mean_fear_imputed=3.6, mean_sen_imputed=4.2, chest-clean_imputed=30, fearfulness_category='medium'),
 Row(p_id_imputed=171, age_imputed=18, fem

In [84]:
# conversion from categorical to numerical
indexer = StringIndexer(inputCol='fearfulness_category', outputCol='fearfulness_category_numeric').fit(df_with_means)
df_with_means = indexer.transform(df_with_means)
df_with_means = df_with_means.drop('mean_fear_imputed', 'fearfulness_category')
# Split the data into training and testing sets
df_train, df_test = df_with_means.randomSplit([0.8, 0.2], seed=42)
X_train = df_train.drop('fearfulness_category_numeric')
X_test = df_test.drop('fearfulness_category_numeric')
y_train = df_train.select('fearfulness_category_numeric')
y_test = df_train.select('fearfulness_category_numeric')

In [87]:
#Add label
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC


train_data = df_train.withColumn("label", df_train["fearfulness_category_numeric"]).drop('fearfulness_category_numeric')
test_data = df_test.withColumn("label", df_test["fearfulness_category_numeric"]).drop('fearfulness_category_numeric')


[Row(p_id_imputed=100, age_imputed=19, female_imputed=0, ethnicity_imputed=1, grip_imputed=58, mean_anx_imputed=4.2, mean_dep_imputed=4.7, mean_sen_imputed=4.4, chest-clean_imputed=60, label=2.0),
 Row(p_id_imputed=101, age_imputed=21, female_imputed=0, ethnicity_imputed=2, grip_imputed=57, mean_anx_imputed=3.7, mean_dep_imputed=3.6, mean_sen_imputed=4.0, chest-clean_imputed=67, label=2.0),
 Row(p_id_imputed=103, age_imputed=21, female_imputed=0, ethnicity_imputed=8, grip_imputed=52, mean_anx_imputed=4.9, mean_dep_imputed=4.0, mean_sen_imputed=3.6, chest-clean_imputed=31, label=2.0),
 Row(p_id_imputed=104, age_imputed=19, female_imputed=0, ethnicity_imputed=1, grip_imputed=53, mean_anx_imputed=3.8, mean_dep_imputed=2.0, mean_sen_imputed=3.6, chest-clean_imputed=52, label=1.0),
 Row(p_id_imputed=105, age_imputed=18, female_imputed=0, ethnicity_imputed=1, grip_imputed=38, mean_anx_imputed=4.1, mean_dep_imputed=3.1, mean_sen_imputed=4.0, chest-clean_imputed=42, label=0.0),
 Row(p_id_imput

In [89]:
#Add feature
inputCols=['age_imputed', 'female_imputed', 'ethnicity_imputed', 'grip_imputed', 'chest-clean_imputed', 'mean_anx_imputed', 'mean_dep_imputed', "mean_sen_imputed"],
outputCol='features'
# Assemble the features into a vector column named "features"
assembler = VectorAssembler(
    inputCols=['age_imputed', 'female_imputed', 'ethnicity_imputed', 'grip_imputed', 'chest-clean_imputed', 'mean_anx_imputed', 'mean_dep_imputed', "mean_sen_imputed"],
    outputCol='features')

train_data_trans = assembler.transform(train_data)
test_data_trans = assembler.transform(test_data)


train_data_trans.collect()

[Row(p_id_imputed=100, age_imputed=19, female_imputed=0, ethnicity_imputed=1, grip_imputed=58, mean_anx_imputed=4.2, mean_dep_imputed=4.7, mean_sen_imputed=4.4, chest-clean_imputed=60, label=2.0, features=DenseVector([19.0, 0.0, 1.0, 58.0, 60.0, 4.2, 4.7, 4.4])),
 Row(p_id_imputed=101, age_imputed=21, female_imputed=0, ethnicity_imputed=2, grip_imputed=57, mean_anx_imputed=3.7, mean_dep_imputed=3.6, mean_sen_imputed=4.0, chest-clean_imputed=67, label=2.0, features=DenseVector([21.0, 0.0, 2.0, 57.0, 67.0, 3.7, 3.6, 4.0])),
 Row(p_id_imputed=103, age_imputed=21, female_imputed=0, ethnicity_imputed=8, grip_imputed=52, mean_anx_imputed=4.9, mean_dep_imputed=4.0, mean_sen_imputed=3.6, chest-clean_imputed=31, label=2.0, features=DenseVector([21.0, 0.0, 8.0, 52.0, 31.0, 4.9, 4.0, 3.6])),
 Row(p_id_imputed=104, age_imputed=19, female_imputed=0, ethnicity_imputed=1, grip_imputed=53, mean_anx_imputed=3.8, mean_dep_imputed=2.0, mean_sen_imputed=3.6, chest-clean_imputed=52, label=1.0, features=Den

In [90]:
from pyspark.ml.classification import OneVsRest

# Train the SVM classifier
svm = LinearSVC(maxIter=10, regParam=0.1)
# set the label column to 'label'
svm.setLabelCol("label")

# set the features column to 'features'
svm.setFeaturesCol("features")

# create an instance of the OneVsRest class
ovr = OneVsRest(classifier=svm)

# fit the OneVsRest model on the training data
ovr_model = ovr.fit(train_data_trans)

y_pred = ovr_model.transform(test_data_trans)

In [91]:
# Evaluate the performance of the classifier
# Predict on the test data using the SVM model



evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
accuracy = evaluator.evaluate(y_pred)
f1_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
f1 = f1_evaluator.evaluate(y_pred)

print(f'Accuracy: {accuracy}')
print(f'F1-score: {f1}')

Accuracy: 0.41379310344827586
F1-score: 0.3581191222570533
